# GIGO Starter Notebook – Data Quality Pipeline (Template)

This is a **starter notebook** for practicing the GIGO (Garbage In, Garbage Out) concept.

Author: **Amantha Bhaskarabhatla**

NUID: **002300618**

You will:

1. Inspect a **dirty transaction dataset**
2. Define a simple **data-quality contract** (rules)
3. Implement:
   - a **data quality report** function
   - a **cleaning pipeline**
4. Compare a simple business metric **before vs after cleaning**

Keep an eye on the To-do comments, as those are the ones you need to finish.


In [1]:
# Basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 4)
plt.rcParams["axes.grid"] = True

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 20)


### Step 1 – Generate a Base (Mostly Clean) Transaction Dataset

We simulate a simple dataset with:

- `customer_id`
- `age`
- `country`
- `product_category`
- `transaction_amount`

This base data is "reasonable" before we inject any garbage.


In [2]:
np.random.seed(42)

n_rows = 500

customer_ids = np.random.randint(1000, 2000, size=n_rows)
ages = np.random.randint(18, 80, size=n_rows)
countries = np.random.choice(
    ["US", "UK", "IN", "DE", "CA"],
    size=n_rows,
    p=[0.3, 0.2, 0.2, 0.15, 0.15],
)
product_categories = np.random.choice(
    ["Electronics", "Clothing", "Grocery", "Beauty"],
    size=n_rows,
)
transaction_amounts = np.round(
    np.random.exponential(scale=50, size=n_rows) + 10,
    2,
)

base_df = pd.DataFrame(
    {
        "customer_id": customer_ids,
        "age": ages,
        "country": countries,
        "product_category": product_categories,
        "transaction_amount": transaction_amounts,
    }
)

base_df.head()


,customer_id,age,country,product_category,transaction_amount
0,1102,20,UK,Grocery,34.92
1,1435,57,DE,Clothing,42.69
2,1860,63,CA,Electronics,55.82
3,1270,41,US,Clothing,27.24
4,1106,67,IN,Clothing,69.37


## Step 2 – Inject Garbage (Make the Data Messy)

Now we **intentionally corrupt** the dataset to simulate real-world dirty data:

- Missing ages and countries  
- Impossible ages  
- Negative and extreme transaction amounts  
- Invalid product categories  
- Duplicate rows  


In [3]:
df = base_df.copy()

# Missing ages
missing_age_idx = np.random.choice(df.index, size=20, replace=False)
df.loc[missing_age_idx, "age"] = np.nan

# Impossible ages
df.loc[np.random.choice(df.index, size=5, replace=False), "age"] = -5
df.loc[np.random.choice(df.index, size=5, replace=False), "age"] = 150

# Missing countries
missing_country_idx = np.random.choice(df.index, size=15, replace=False)
df.loc[missing_country_idx, "country"] = np.nan

# Invalid product categories
df.loc[np.random.choice(df.index, size=10, replace=False), "product_category"] = "UnknownCategory"

# Negative transaction amounts
df.loc[np.random.choice(df.index, size=8, replace=False), "transaction_amount"] *= -1

# Extreme outliers
df.loc[np.random.choice(df.index, size=5, replace=False), "transaction_amount"] = (
    df["transaction_amount"].max() * 20
)

# Duplicate rows
duplicates = df.sample(10, random_state=1)
df = pd.concat([df, duplicates], ignore_index=True)

df.head()


,customer_id,age,country,product_category,transaction_amount
0,1102,NaN,UK,Grocery,34.92
1,1435,57.0,DE,Clothing,42.69
2,1860,63.0,CA,Electronics,55.82
3,1270,41.0,US,Clothing,27.24
4,1106,67.0,IN,Clothing,69.37


In [4]:
print("Shape:", df.shape)
print("\nInfo:")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum())

print("\nNumber of duplicated rows:", df.duplicated().sum())


Shape: (510, 5)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         510 non-null    int64  
 1   age                 490 non-null    float64
 2   country             495 non-null    object 
 3   product_category    510 non-null    object 
 4   transaction_amount  510 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 20.1+ KB
None

Missing values per column:
customer_id            0
age                   20
country               15
product_category       0
transaction_amount     0
dtype: int64

Number of duplicated rows: 10


In [5]:
print("Age stats:")
display(df["age"].describe())

print("\nTransaction amount stats:")
display(df["transaction_amount"].describe())

print("\nUnique countries:", df["country"].unique())
print("Unique product categories:", df["product_category"].unique())


Age stats:


,age
count,490.000000
mean,50.410204
std,21.194713
min,-5.000000
25%,34.000000
50%,50.000000
75%,66.000000
max,150.000000



Transaction amount stats:


,transaction_amount
count,510.000000
mean,132.639941
std,750.000652
min,-185.180000
25%,22.490000
50%,43.720000
75%,75.717500
max,7641.800000



Unique countries: ['UK' 'DE' 'CA' 'US' 'IN' nan]
Unique product categories: ['Grocery' 'Clothing' 'Electronics' 'Beauty' 'UnknownCategory']


## Step 3 – Define Validation Rules (To-Do)

Now define what **valid** data means for this dataset.

For example:

- `age`: between 18 and 100  
- `country`: one of {US, UK, IN, DE, CA}  
- `product_category`: one of {Electronics, Clothing, Grocery, Beauty}  
- `transaction_amount`: > 0 and ≤ 1000  

You will write a **`validation_rules`** dictionary that encodes these assumptions.


In [6]:
# TODO: define validation rules for each key column:
# - age: {"min": 18, "max": 100}
# - transaction_amount: {"min": 0, "max": 1000}
# - country: {"allowed": [...]}
# - product_category: {"allowed": [...]}

validation_rules = {
    # "age": {...},
    # "transaction_amount": {...},
    # "country": {...},
    # "product_category": {...},
}

validation_rules


{}

## Step 4 – Implement a Data Quality Report

We want a function:

```python
data_quality_report(df, validation_rules)
```

that returns a small table with, for each column:

- missing_pct

- invalid_pct

Invalid means:

- Out of allowed range (for numeric columns), or

- Not in the allowed list (for categorical columns).

In [ ]:



def data_quality_report(df: pd.DataFrame, rules: dict) -> pd.DataFrame:
    """
    For each relevant column (age, country, product_category, transaction_amount):
    - Compute missing_pct (percent of rows that are NaN)
    - Compute invalid_pct (percent of rows that violate the rules)
    Return a DataFrame with one row per column.
    """
    total_rows = len(df)
    records = []

    # TODO: loop over each column in `rules` and populate `records`

    report_df = pd.DataFrame(records)
    return report_df


    #TODO: implement logic similar to:
    # - age: out of [min, max] -> invalid
    # - country: not in allowed list -> invalid
    # - product_category: not in allowed list -> invalid
    # - transaction_amount: <= min or > max -> invalid

    # Example structure for one column (you will adapt it):
    #
    # col = "age"
    # missing = df[col].isna().sum()
    # invalid_mask = <some boolean condition>
    # invalid = invalid_mask.sum()
    # records.append({
    #     "column": col,
    #     "missing_pct": missing / total_rows * 100,
    #     "invalid_pct": invalid / total_rows * 100,
    # })

    report_df = pd.DataFrame(records)
    return report_df


SyntaxError: invalid syntax (ipython-input-662580982.py, line 1)

In [ ]:
# TODO: once your function is implemented and validation_rules is filled,
# run the data quality report on the dirty df.

dq_before = data_quality_report(df, validation_rules)
dq_before


## Step 5 – Implement a Simple Cleaning Pipeline

Implement a function:

```python
clean_df = clean_data(df, validation_rules)
```

that:

1. Drops duplicate rows

2. For age:

- Values outside [min, max] → set to NaN

- Then fill NaN with the median age

3. For country:

- Values not in allowed list → NaN

- Then fill NaN with the mode (most frequent) country

4. For product_category:

- Values not in allowed list → NaN

- Then fill NaN with the mode category

5. For transaction_amount:

- Values ≤ min or > max → NaN

- Then fill NaN with the median transaction amount

In [ ]:



### 🔹 Cell 15 — (Code) Cleaning Function (TODO)


def clean_data(df: pd.DataFrame, rules: dict) -> pd.DataFrame:
    """
    TODO:
    - Copy the input df
    - Drop duplicates
    - Fix age, country, product_category, transaction_amount
      according to the rules and strategy described.
    - Return the cleaned DataFrame.
    """
    clean_df = df.copy()

    # TODO: 1) drop duplicates
    # clean_df = ...

    # TODO: 2) clean age (out of range -> NaN -> fill with median)

    # TODO: 3) clean country (invalid -> NaN -> fill with mode)

    # TODO: 4) clean product_category (invalid -> NaN -> fill with mode)

    # TODO: 5) clean transaction_amount (invalid -> NaN -> fill with median)

    return clean_df


In [ ]:
# TODO: once clean_data is implemented, run it:
clean_df = clean_data(df, validation_rules)

# Data quality report AFTER cleaning
dq_after = data_quality_report(clean_df, validation_rules)
dq_after


## Step 6 – Compare a Simple Business Metric (Before vs After)

We will use:

> **Average transaction amount by country**

Compute this:

1. On the **dirty** `df`  
2. On the **clean** `clean_df`  

Then compare:

- Are the averages very different?
- Would a manager be misled if they only saw the dirty version?


In [ ]:
# TODO:
# 1. Compute avg transaction_amount by country on df (dirty)
avg_before = df.groupby("country")["transaction_amount"].mean()

# 2. Compute avg transaction_amount by country on clean_df
avg_after = clean_df.groupby("country")["transaction_amount"].mean()

print("Average transaction amount by country (BEFORE cleaning):")
display(avg_before)

print("\nAverage transaction amount by country (AFTER cleaning):")
display(avg_after)

# Optional: plot side-by-side if you like
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

avg_before.plot(kind="bar", ax=axes[0], title="Before Cleaning")
axes[0].set_ylabel("Avg Transaction Amount")

avg_after.plot(kind="bar", ax=axes[1], title="After Cleaning")

plt.tight_layout()
plt.show()


## Step 7 – Reflection

Answer in a few bullet points:

- Which columns had the most issues **before** cleaning?
- Did the data-quality report look better **after** cleaning?
- Did any country's average transaction amount change a lot?
- What is one concrete way this GIGO effect could mislead a real business?

You can now go back and:
- Tighten or relax the validation rules  
- Add new columns (like `loyalty_score`)  
- Extend your data-quality report and cleaning pipeline  
to explore GIGO even more deeply.
